In [ ]:
import requests
import os
import pandas
import geopandas
from shapely.geometry import Point

### Loop Through for Avg, Max, and Min Temps

In [ ]:
#Define URL from NDAWN API
variablelist = ['ddavt','ddmxt','ddmnt']
#Loop through variables
for temp in variablelist:
    ndurl = fr'https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&variable={temp}&ttype=daily&quick_pick=30_d'
    #Define Name and Location of New CSV
    ndoutput = fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\NDAWN_30Day_{temp}_Raw.csv"
    #Pulls CSV from URL
    ndresponse = requests.get(ndurl, stream=True)
    #Writing CSV to file on local drive if status code is correct.
    if ndresponse.status_code == 200:
        with open(ndoutput, 'wb') as file:
            file.write(ndresponse.content)

    #Reads CSV file
    nddataframe = pandas.read_csv(fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\NDAWN_30Day_{temp}_Raw.csv", skiprows=4, index_col=False)
    #Defines new names for CSV Headers
    new_names = ['Location', 'Lat', 'Long', 'Elevation ft', 'Year', 'Month', 'Day', f'{temp}', 'Flag']
    nddataframe.columns = new_names
    #Saves New Names to CSV
    nddataframe.to_csv(fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\NDAWN_30Day_{temp}.csv", index=False) 
    
     # Read the CSV file into a DataFrame, Information on how to do the groupby provided by OpenAI,2023
    nddataframe = pandas.read_csv(fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\NDAWN_30Day_{temp}.csv")

    # Group the columns with the same station names and average the temp
    ndresult = nddataframe.groupby(['Lat','Long','Location'])[f'{temp}'].mean().reset_index()
    
    #Save to CSV
    ndresult.to_csv(fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\NDAWN_30Day_{temp}_Avg.csv", index=False)
    #Read CSV 
    ndresult = pandas.read_csv(fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\NDAWN_30Day_{temp}_Avg.csv")
    #Create points with Lat and long coordinates
    ndgeometry = [Point(xy) for xy in zip(ndresult['Long'], ndresult['Lat'])]
    ndgeoframe = geopandas.GeoDataFrame(ndresult, geometry=ndgeometry, crs='4326')
    #Write geometry to shapefile
    ndgeoframe.to_file(fr'\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\ND30Day_{temp}.shp') 

### Loop Through Points for Different Interpolation Methods

In [ ]:
#List of Temp Variables
variablelist = ['ddavt','ddmxt','ddmnt']
#Loop through Variables
for temp in variablelist:
    #Inverse Distance Weighting
    arcpy.ddd.Idw(
        in_point_features=fr'\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\ND30Day_{temp}.shp',
        z_field=temp,
        out_raster=fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3-2.gdb\IDW_{temp}",
        power=2,
        in_barrier_polyline_features=None
    )
    #Empiricial Bayesian Kriging
    arcpy.ga.EmpiricalBayesianKriging(
        in_features=fr'\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\ND30Day_{temp}.shp',
        z_field=temp,
        out_ga_layer=None,
        out_raster=fr'\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3-2.gdb\Kriging_{temp}',
    )
    #Diffusion Interpolation
    arcpy.ga.DiffusionInterpolationWithBarriers(
        in_features=fr'\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3Data\ND30Day_{temp}.shp',
        z_field=temp,
        out_raster=fr"\\Mac\Home\Documents\ArcGIS\Projects\Lab3-2\Lab3-2.gdb\Diffusion_{temp}",
        number_iterations=100,
    )